In [2]:
import pandas as pd
import glob

path = r'.\scrapy\larvf\larvf_0.01_output\*.json'

L_files = list()
for filename in glob.glob(path):
    DF_part = pd.read_json(filename)
    L_files.append(DF_part)

DF_rvf0 = pd.concat(L_files, axis=0, ignore_index=True)
print(DF_rvf0.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169460 entries, 0 to 169459
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   domain_name         169460 non-null  object
 1   domain_stars        169460 non-null  int64 
 2   wine_name           169460 non-null  object
 3   Région              168997 non-null  object
 4   Appellation         169455 non-null  object
 5   vintage_twitterimg  169460 non-null  object
 6   wine_types          169460 non-null  object
 7   wine_varieties      169460 non-null  object
 8   vintage             169460 non-null  object
 9   vintage_types       20242 non-null   object
 10  vintage_varieties   18609 non-null   object
 11  vintage_prices      169460 non-null  object
 12  review_score        169460 non-null  object
 13  review_topline      169460 non-null  object
 14  review_text         169460 non-null  object
dtypes: int64(1), object(14)
memory usage: 19.4+ MB
None

In [3]:
import re

maincol = ['domain_name', 'wine_name', 'vintage', 'review_score', 'review_topline']

# Construit une version numérique des notes (x/20) :
DF_rvf = DF_rvf0.copy()
DF_rvf['review_score_float'] = DF_rvf.review_score.apply(
    lambda s : float(n) if ((n := re.sub(r"/.*$", "", s).replace(",", ".")) != "") else None
)

# Les notes des "Guides rouges" (guide des vins pas chers) ne sont pas fiables ou pas à la
# même échelle que celles du guide vert, et sans intérêt pour l'analyse des vins réputés,
# donc je les supprime :
keepmask = (DF_rvf.review_topline.str.slice(0,11)!='Guide rouge')
DF_rvf = DF_rvf[keepmask]
display(DF_rvf.sort_values(by='review_score_float', ascending=False)[maincol].head(20))

,domain_name,wine_name,vintage,review_score,review_topline
53661,Domaines David Duband - François Feuillet,Chambertin Grand Cru,2013,20/20,Guide vert 2016
37050,Château Climens,Barsac,1997,20/20,Guide vert 2007
104146,Château Mouton Rothschild,Pauillac,2000,20/20,Guide vert 2006
394,Domaine d’Auvenay,Mazis-Chambertin Grand Cru,2009,20/20,Guide vert 2012
393,Domaine d’Auvenay,Mazis-Chambertin Grand Cru,2010,20/20,Guide vert 2013
104147,Château Mouton Rothschild,Pauillac,2000,20/20,Guide vert 2005
391,Domaine d’Auvenay,Mazis-Chambertin Grand Cru,2012,20/20,Guide vert 2015
390,Domaine d’Auvenay,Mazis-Chambertin Grand Cru,2013,20/20,Guide vert 2016
389,Domaine d’Auvenay,Mazis-Chambertin Grand Cru,2014,20/20,Guide vert 2017
104148,Château Mouton Rothschild,Pauillac,2000,20/20,Guide vert 2004


In [4]:
""" display(DF_rvf.sort_values(by='review_score_float', ascending=True)[maincol].head(30))
display(DF_rvf.review_score_float.dropna().value_counts(sort=False).sort_index())
display(DF_rvf[maincol][DF_rvf.review_score_float==10].head(30))
display(DF_rvf[maincol][DF_rvf.review_score_float==11].head(30)) """

# Les notes les plus faibles (<11) sont vraisemblablement, pour la plupart, des erreurs, 
# donc je les supprime :
DF_rvf = DF_rvf[DF_rvf.review_score_float>=11]
display(DF_rvf.review_score_float.dropna().value_counts(sort=False).sort_index())


11.0       61
11.5       12
12.0     3968
12.5      567
13.0     4971
13.5     3579
14.0    20857
14.5    10078
15.0    23251
15.5    12289
16.0    27025
16.5     9141
17.0    12630
17.5     3417
18.0     6010
18.5     1483
19.0     2814
19.5      370
20.0      377
Name: review_score_float, dtype: int64

In [5]:
DF_rvf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142900 entries, 2 to 169458
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   domain_name         142900 non-null  object 
 1   domain_stars        142900 non-null  int64  
 2   wine_name           142900 non-null  object 
 3   Région              142567 non-null  object 
 4   Appellation         142895 non-null  object 
 5   vintage_twitterimg  142900 non-null  object 
 6   wine_types          142900 non-null  object 
 7   wine_varieties      142900 non-null  object 
 8   vintage             142900 non-null  object 
 9   vintage_types       20155 non-null   object 
 10  vintage_varieties   18551 non-null   object 
 11  vintage_prices      142900 non-null  object 
 12  review_score        142900 non-null  object 
 13  review_topline      142900 non-null  object 
 14  review_text         142900 non-null  object 
 15  review_score_float  142900 non-nul

In [6]:
# Les régions non renseignées correspondent à des vrais vins :
DF_rvf[DF_rvf['Région'].isna()].sample(frac=1).head(15)


,domain_name,domain_stars,wine_name,Région,Appellation,vintage_twitterimg,wine_types,wine_varieties,vintage,vintage_types,vintage_varieties,vintage_prices,review_score,review_topline,review_text,review_score_float
164780,Château Vignelaure,0,VDP du Var Domaine de Vignelaure,NaN,Côtes de Provence,bouteille-vin-provence-rouge.jpg,,,1997,None,None,,12/20,Guide vert 2001,,12.0
153441,Château Saint-Georges,0,Saint-Georges Saint-Émilion,NaN,Saint-Georges-Saint-Émilion,bouteille-vin-bordeaux-rouge.jpg,,,1995,None,None,,14/20,Guide vert 2001,,14.0
137744,Château Romanin,0,VDP des Bouches du Rhône Jean Le Troubadour,NaN,Côtes de Provence,bouteille-vin-provence-rouge.jpg,,,2000,None,None,28 €,15/20,Guide vert 2002,,15.0
46040,Domaine Cazes,0,VDP Côtes Catalanes Le Credo,NaN,Côtes du Roussillon,bouteille-vin-roussillon-rouge.jpg,,,2000,None,None,"13,70 €",15/20,Guide vert 2005,,15.0
77355,Domaine Gauby,3,VDP Coteaux de Fenouillèdes Le Soula,NaN,Côtes du Roussillon,bouteille-vin-roussillon-blanc.jpg,,,2001,None,None,,17/20,Guide vert 2004,,17.0
29602,Domaine Le Conte des Floris,1,Cartagène Noir Rimage,NaN,Vin de Liqueur (Languedoc),bouteille-vin-languedoc-rouge.jpg,,,2009,None,None,,15/20,Guide vert 2012,,15.0
77352,Domaine Gauby,3,VDP Côtes Catalanes Aleaux,NaN,Côtes du Roussillon,bouteille-vin-roussillon-blanc.jpg,,,2000,None,None,,16/20,Guide vert 2002,,16.0
66220,Domaine Ferrer-Ribière,0,VDP Côtes Catalanes Syrahnosaurus Rex,NaN,Côtes du Roussillon,bouteille-vin-roussillon-rouge.jpg,,,2003,None,None,15 €,15/20,Guide vert 2005,,15.0
164800,Château Vignelaure,0,VDP des Coteaux du Verdon La Colline,NaN,Côtes de Provence,bouteille-vin-provence-rouge.jpg,,,1999,None,None,24 €,14/20,Guide vert 2004,,14.0
126131,Julien Pilon,1,IGP Collines Rhodaniennes De l'Autre Rive,NaN,IGP Collines Rhodaniennes,bouteille-vin-vallee-rhone-rouge.jpg,,,2014,None,None,14 €,"13,5/20",Guide vert 2016,,13.5


In [7]:
# Les appellations non renseignées correspondent à de vrais vins :
DF_rvf[DF_rvf['Appellation'].isna()].sample(frac=1).head(15)

,domain_name,domain_stars,wine_name,Région,Appellation,vintage_twitterimg,wine_types,wine_varieties,vintage,vintage_types,vintage_varieties,vintage_prices,review_score,review_topline,review_text,review_score_float
34939,Clos Nicrosi,1,Le Rappu,Provence,NaN,vin-rvf.jpg,"Vin de liqueur, Moelleux",Aleatico (Rouge) 100%,,"Vin de liqueur, Moelleux",Aleatico (Rouge) 100%,,"16,5/20",Guide vert 2017,"Confiture de figue et épices douces, une touch...",16.5
25444,Christophe Curtat,1,IGP Roussanne,Vallée du Rhône,NaN,vin-rvf.jpg,,,2014,None,None,,"15,5/20",Guide vert 2017,"Une belle fraîcheur se dessine, une roussanne ...",15.5
25447,Christophe Curtat,1,IGP Syrah,Vallée du Rhône,NaN,vin-rvf.jpg,,,2014,None,None,,"14,5/20",Guide vert 2017,"Une palette sur le fruit, simple mais lisible,...",14.5
44671,Château de Chamilly,1,Montagny Premier Cru Les Burnins,Bourgogne,NaN,vin-rvf.jpg,,,2014,None,None,,"14,5/20",Guide vert 2017,Des notes de fleurs de vignes. La bouche est d...,14.5
153915,Domaine Roger Sabon,1,VDF Plaisir,Vallée du Rhône,NaN,vin-rvf.jpg,,,2015,None,None,,"13,5/20",,Un beau VDF comme on aimerait en trouver plus ...,13.5


In [8]:
# Renommage des colonnes 'Région' et 'Appellation' pour harmoniser :
DF_rvf = DF_rvf.rename({'Région': 'region', 'Appellation': 'appellation'}, axis=1)
DF_rvf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142900 entries, 2 to 169458
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   domain_name         142900 non-null  object 
 1   domain_stars        142900 non-null  int64  
 2   wine_name           142900 non-null  object 
 3   region              142567 non-null  object 
 4   appellation         142895 non-null  object 
 5   vintage_twitterimg  142900 non-null  object 
 6   wine_types          142900 non-null  object 
 7   wine_varieties      142900 non-null  object 
 8   vintage             142900 non-null  object 
 9   vintage_types       20155 non-null   object 
 10  vintage_varieties   18551 non-null   object 
 11  vintage_prices      142900 non-null  object 
 12  review_score        142900 non-null  object 
 13  review_topline      142900 non-null  object 
 14  review_text         142900 non-null  object 
 15  review_score_float  142900 non-nul

In [9]:
# Enregistrement :
DF_rvf.to_csv('clean_data\larvf.csv', index=False)